In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import baostock as bs
bs.login()
result = bs.query_history_k_data('sh.601318', 
                                 fields = 'date,open,high, low,close,volume',
                                 start_date = '2020-01-01',
                                 end_date = '2021-05-01',
                                 frequency='d')
dataset = result.get_data().set_index('date').applymap(lambda x: float(x))
bs.logout()
dataset['Open_Close'] = (dataset['open'] - dataset['close'])/dataset['open']
dataset['High_Low'] = (dataset['high'] - dataset['low'])/dataset['low']
dataset['Increase_Decrease'] = np.where(dataset['volume'].shift(-1) > dataset['volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['open'].shift(-1) > dataset['open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['close'].shift(-1) > dataset['close'],1,0)
dataset['Returns'] = dataset['close'].pct_change()
dataset = dataset.dropna()
dataset['Up_Down'] = np.where(dataset['Returns'].shift(-1) > dataset['Returns'],'Up','Down')
dataset = dataset.dropna()
dataset.head()

login success!
logout success!


,open,high,low,close,volume,Open_Close,High_Low,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns,Up_Down
date,,,,,,,,,,,,
2020-01-03,86.81,86.88,85.90,86.20,59498001.0,0.007027,0.011409,1,0,0,0.000929,Down
2020-01-06,85.92,86.87,85.50,85.60,63644804.0,0.003724,0.016023,0,1,1,-0.006961,Up
2020-01-07,86.01,86.46,85.67,86.15,45218832.0,-0.001628,0.009221,1,0,0,0.006425,Down
2020-01-08,85.98,85.98,84.50,85.00,62805311.0,0.011398,0.017515,0,0,1,-0.013349,Up
2020-01-09,85.58,85.61,84.69,85.04,59871202.0,0.006310,0.010863,0,0,0,0.000471,Down


In [6]:
import plotly.express as px

df = dataset
fig=px.scatter(df, x="open", y="close", 
                 trendline="ols")
fig.show()

In [7]:
import plotly.express as px
fig=px.scatter(df, x="open", y="close", 
              facet_col="Increase_Decrease", 
              color="Up_Down", trendline="ols")
fig.show()

In [8]:
import plotly.express as px
results = px.get_trendline_results(fig)
results.query(
  "Up_Down == 'Up' and Increase_Decrease == '1'"
       ).px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                     2634.
Date:                Fri, 09 Sep 2022   Prob (F-statistic):           1.90e-58
Time:                        14:18:17   Log-Likelihood:                -94.626
No. Observations:                  74   AIC:                             193.3
Df Residuals:                      72   BIC:                             197.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.9611      1.429      3.471      0.001       2.112       7.811
x1             0.9331      0.018     51.321      0.000       0.897       0.969
==============================================================================
Omnibus:                        2.158   Durbin-Watson:                   1.663
Prob(Omnibus):                  0.340   Jarque-Bera (JB):                1.593
Skew:                          -0.083   Prob(JB):                        0.451
Kurtosis:                       3.699   Cond. No.                     1.10e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.1e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## 线性回归可视化

可以使用Scikit-learn的线性回归执行相同的预测。与直接用plotly.express拟合普通最小二乘回归不同，这是通过散点图和拟合线组合的方式绘制图形，这会更加灵活，除了添加普通线性回归拟合曲线，还可以组合其他线性回归曲线，即将拟合结果很好地可视化出来。

In [12]:
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
X = df.open.values.reshape(-1, 1)
# 回归模型训练
model = LinearRegression()
model.fit(X, df.close)
# 生产预测点
x_range = np.linspace(X.min(), X.max(), 100)
y_range = model.predict(x_range.reshape(-1, 1))
# 图形绘制
fig = px.scatter(df, x='open', y='close', opacity=0.65)
fig.add_traces(go.Scatter(x=x_range, y=y_range, name='Regression Fit'))
fig.show()